In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import kerastuner as kt
from sklearn.model_selection import train_test_split
import seaborn as sns


def plot_curve(epochs, hist, list_of_metrics,name):
    """Plot a curve of one or more classification metrics vs. epoch."""  
    plt.figure()
    plt.xlabel("Epoch "+name)
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)
    plt.legend()
    
    
# for activation functions check https://keras.io/api/layers/activations/
def create_model2(my_learning_rate,momentum,layers,my_metrics,my_act_function = "softmax"):
    model = tf.keras.models.Sequential()
    #model.add(my_feature_layer)

    for layer in layers:
        model.add(tf.keras.layers.Dense(units = layer, activation = my_act_function))
    model.add(tf.keras.layers.Dense(units=15,name='Output', activation = 'relu'))                             
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=my_learning_rate,momentum=momentum),                                       
                loss=tf.keras.losses.MeanAbsoluteError(),
                metrics=my_metrics)
    return model


def train_model(model,x_data, y_data, epochs, label_name,
                batch_size=None,shuffle=True):
    #features = {name:np.array(value) for name, value in dataset.items()}
    history = model.fit(x=x_data, y=y_data, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle,validation_split=0.2,
                       callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])
  
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist
    
    
#returns dataframe
def test_model(model,x_data, y_data ,label_name):
    evaluation=model.evaluate(x = x_data, y = y_data, batch_size=150)
    predicted = model.predict(x_data)
    df_test=pd.DataFrame(y_data,columns=[label_name])
   # print(predicted)
    df_predict=pd.DataFrame(predicted,columns=[label+"_pred" for label in label_name])
    return pd.concat([df_test,df_predict], axis=1)



In [27]:

all_labels_features=["all_maxima","Intensity","overlap_s0_s2_k6a","overlap_s0_s2_k1","overlap_s0_s2_k9a","k6a1","k6a2","k11","k12","k9a1","k9a2","delta","lambda"]
all_data=pd.read_csv("G:\OneDrive - bwedu\Master\Forschungspraktikum\Inga\pc-forschi\generated_Data/all_param_4_values_with_overlap.csv")
df_feature_labels=all_data[all_labels_features]

max_no_of_peak_list=max(all_data["no_of_max"])
print(max_no_of_peak_list)
df_feature_labels.head()

7


,all_maxima,Intensity,overlap_s0_s2_k6a,overlap_s0_s2_k1,overlap_s0_s2_k9a,k6a1,k6a2,k11,k12,k9a1,k9a2,delta,lambda
0,[0.466 0.54 0.612 0.67 0.742],[104.475076 104.507572 87.7231385 58.12209...,"[0.5232, 0.3386, 0.1098, 0.0239, 0.004]","[0.7501, 0.2155, 0.031, 0.003, 0.0002]","[0.8102, 0.1704, 0.018, 0.0013, 0.0001]",-0.1,-0.1,-0.1,-0.1,-0.1,-0.1000,0.6000,0.1000
1,[0.51 0.584 0.65 0.712 0.778],[121.232278 120.055307 85.8745546 56.17655...,"[0.5232, 0.3386, 0.1098, 0.0239, 0.004]","[0.7501, 0.2155, 0.031, 0.003, 0.0002]","[0.9769, 0.0228, 0.0003, 0.0, 0.0]",-0.1,-0.1,-0.1,-0.1,-0.1,-0.0333,0.6000,0.2333
2,[0.482 0.556 0.628 0.686 0.756],[101.883081 101.900808 85.5862194 56.79913...,"[0.5232, 0.3386, 0.1098, 0.0239, 0.004]","[0.7501, 0.2155, 0.031, 0.003, 0.0002]","[0.8102, 0.1704, 0.018, 0.0013, 0.0001]",-0.1,-0.1,-0.1,-0.1,-0.1,-0.1000,0.6000,0.2333
3,[0.418 0.492 0.564 0.622 0.692],[101.197996 101.175315 85.035938 56.43656...,"[0.5232, 0.3386, 0.1098, 0.0239, 0.004]","[0.7501, 0.2155, 0.031, 0.003, 0.0002]","[0.8102, 0.1704, 0.018, 0.0013, 0.0001]",-0.1,-0.1,-0.1,-0.1,-0.1,-0.1000,0.5333,0.2333
4,[0.4 0.474 0.546 0.604 0.676],[104.345828 104.372914 87.6137903 58.09441...,"[0.5232, 0.3386, 0.1098, 0.0239, 0.004]","[0.7501, 0.2155, 0.031, 0.003, 0.0002]","[0.8102, 0.1704, 0.018, 0.0013, 0.0001]",-0.1,-0.1,-0.1,-0.1,-0.1,-0.1000,0.5333,0.1000


In [28]:
#this returns training and testing for model_param_from_overlap


#convert string lists into numpy arrays in dict

all_maxima_array=np.asarray([  np.asarray([x for x in row.replace("[","").replace("]","").replace("  "," ",5).replace(" ",";").split(";") if x!=""],dtype=np.float64)     for row in df_feature_labels["all_maxima"] ])

intensity_array=np.asarray([  np.asarray([x for x in row.replace("[","").replace("]","").replace("  "," ",5).replace(" ",";").split(";") if x!=""],dtype=np.float64)     for row in df_feature_labels["Intensity"] ])

overlap_s0_s2_k6a_array=np.asarray([  np.asarray([x for x in row.replace(",","").replace("[","").replace("]","").replace("  "," ",5).replace(" ",";").split(";") if x!=""],dtype=np.float64)     for row in df_feature_labels["overlap_s0_s2_k6a"] ])

overlap_s0_s2_k1_array=np.asarray([  np.asarray([x for x in row.replace(",","").replace("[","").replace("]","").replace("  "," ",5).replace(" ",";").split(";") if x!=""],dtype=np.float64)     for row in df_feature_labels["overlap_s0_s2_k1"] ])

overlap_s0_s2_k9a_array=np.asarray([  np.asarray([x for x in row.replace(",","").replace("[","").replace("]","").replace("  "," ",5).replace(" ",";").split(";") if x!=""],dtype=np.float64)     for row in df_feature_labels["overlap_s0_s2_k9a"] ])


#pad all_maxima_array and intensity_array
all_maxima_array_padded=np.zeros((len(all_maxima_array),max_no_of_peak_list))
intensity_array_padded=np.zeros((len(intensity_array),max_no_of_peak_list))

for i in range(len(all_maxima_array)):
    for j in range(len(all_maxima_array[i])):
        all_maxima_array_padded[i][j]=all_maxima_array[i][j]
        intensity_array_padded[i][j]=intensity_array[i][j]

        
        
concat_label=np.concatenate((overlap_s0_s2_k6a_array,overlap_s0_s2_k1_array,overlap_s0_s2_k9a_array),axis=1)
concat_feature=np.concatenate((all_maxima_array_padded,intensity_array_padded),axis=1)
x_train_overlap, x_test_overlap,y_train_overlap,y_test_overlap = train_test_split( concat_feature, concat_label  ,test_size=0.20, random_state=42)

     

H:\Users\Gwydion\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [29]:
#this returns training and testing for model_overlap_from_spectrum
#convert string lists into numpy arrays in dict


In [30]:
model_overlap_from_spectrum = tf.keras.models.load_model('G:\OneDrive - bwedu\Master\Forschungspraktikum\Inga\pc-forschi\machine_learning\Experiments with all param\saved_Models/model_overlap_from_spectrum_err_0_0057')


In [31]:
model_overlap_from_spectrum.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             multiple                  240       
_________________________________________________________________
dense_29 (Dense)             multiple                  4352      
_________________________________________________________________
dense_30 (Dense)             multiple                  131584    
_________________________________________________________________
dense_31 (Dense)             multiple                  1050624   
_________________________________________________________________
dense_32 (Dense)             multiple                  2098176   
_________________________________________________________________
dense_33 (Dense)             multiple                  65600     
_________________________________________________________________
dense_34 (Dense)             multiple                 

In [33]:
all_label_list=['overlap_k6a 0',"overlap_k6a 1","overlap_k6a 2","overlap_k6a 3","overlap_k6a 4","overlap_k1 0","overlap_k1 1","overlap_k1 2","overlap_k1 3","overlap_k1 4","overlap_k9a 0","overlap_k9a 1","overlap_k9a 2","overlap_k9a 3","overlap_k9a 4"]


delta_test_result=test_model(model_overlap_from_spectrum,x_test_overlap,y_test_overlap,all_label_list)

columns_names=delta_test_result.columns

compare_k6a=columns_names[[0,1,2,3,4,15,16,17,18,19]]
compare_k1=columns_names[[5,6,7,8,9,20,21,22,23,24]]
compare_k9a=columns_names[[10,11,12,13,14,25,26,27,28,29]]
#print(compare_k6a)

delta_test_result[compare_k6a].head(50)
#delta_test_result[compare_k1].head(5)
#delta_test_result[compare_k9a].head(5)

88/88 [==============================] - ETA: 0s - loss: 0.0057 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0056 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0057 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0057 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0057 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0056 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0056 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.005 - 1s 9ms/step - loss: 0.0055 - mean_absolute_error: 0.0055


,"(overlap_k6a 0,)","(overlap_k6a 1,)","(overlap_k6a 2,)","(overlap_k6a 3,)","(overlap_k6a 4,)",overlap_k6a 0_pred,overlap_k6a 1_pred,overlap_k6a 2_pred,overlap_k6a 3_pred,overlap_k6a 4_pred
0,0.9307,0.0669,0.0024,0.0001,0.000,0.930712,0.067350,0.002743,0.0,0.0
1,0.9307,0.0669,0.0024,0.0001,0.000,0.932051,0.066253,0.002037,0.0,0.0
2,0.9307,0.0669,0.0024,0.0001,0.000,0.938701,0.062366,0.000000,0.0,0.0
3,0.9307,0.0669,0.0024,0.0001,0.000,0.938760,0.062223,0.000000,0.0,0.0
4,0.9307,0.0669,0.0024,0.0001,0.000,0.938827,0.062188,0.000000,0.0,0.0
5,0.9307,0.0669,0.0024,0.0001,0.000,0.931544,0.066840,0.000000,0.0,0.0
6,0.5232,0.3386,0.1098,0.0239,0.004,0.523986,0.337338,0.107529,0.0,0.0
7,0.9307,0.0669,0.0024,0.0001,0.000,0.932053,0.066145,0.000000,0.0,0.0
8,0.9307,0.0669,0.0024,0.0001,0.000,0.931092,0.067347,0.000000,0.0,0.0
9,0.9307,0.0669,0.0024,0.0001,0.000,0.931982,0.065534,0.000000,0.0,0.0


In [ ]:
#my_model.save("saved_Models/lambda_100_best_model_main_max_err_0_0027")